In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 27693354
paper_name = 'mulleder_ralser_2016' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/mmc3.xlsx', sheet_name='intracellular_concentration_mM')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4678 x 21


In [7]:
original_data.head()

,ORF,gene,alanine,aspartate,glutamate,phenylalanine,glycine,histidine,isoleucine,lysine,...,methionine,asparagine,proline,glutamine,arginine,serine,threonine,valine,tryptophan,tyrosine
0,YAL002W,VPS8,7.575695,6.023942,29.799218,0.172805,1.668835,5.518672,0.346954,1.017763,...,0.148572,1.150852,1.529600,38.743733,5.000892,2.227969,3.830483,1.354447,0.022283,1.064131
1,YAL004W,NaN,5.885958,5.035586,23.627768,0.280726,1.793227,3.916185,0.407873,0.662228,...,0.141925,1.052574,1.165382,27.583021,2.375404,2.284112,3.952990,1.323046,0.040681,3.180600
2,YAL005C,SSA1,5.903218,4.826971,23.024489,0.300006,1.846948,4.764524,0.410972,0.707888,...,0.136524,1.121279,1.149201,22.608953,2.697176,2.152940,3.711750,1.367357,0.043073,2.724967
3,YAL007C,ERP2,5.882356,5.405774,20.391227,0.258860,1.884296,5.554489,0.357562,0.946165,...,0.093918,1.289670,1.161532,28.793732,2.875816,2.414602,3.967596,1.231819,0.038839,2.659337
4,YAL008W,FUN14,5.838249,4.682146,19.302517,0.286509,2.366559,7.124882,0.424865,1.244891,...,0.108243,1.224425,1.125802,26.754974,4.704578,2.784608,3.851613,1.204111,0.055207,1.465688


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, gene, alanine, aspartate, glutamate, phenylalanine, glycine, histidine, isoleucine, lysine, leucine, methionine, asparagine, proline, glutamine, arginine, serine, threonine, valine, tryptophan, tyrosine, orf]
Index: []

[0 rows x 22 columns]


In [12]:
original_data.drop(columns=['ORF','gene'], inplace=True)

In [13]:
original_data.set_index('orf', inplace=True)

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(4677, 19)

In [19]:
dataset_ids = list(np.arange(11788,11807))

# Prepare the final dataset

In [21]:
data = original_data.copy()

In [22]:
datasets = datasets.reindex(index=dataset_ids)

In [23]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [24]:
data.head()

dataset_id,11788,11789,11790,11791,11792,11793,11794,11795,11796,11797,11798,11799,11800,11801,11802,11803,11804,11805,11806
data_type,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,,,,,,,,
YAL002W,7.575695,6.023942,29.799218,0.172805,1.668835,5.518672,0.346954,1.017763,0.140872,0.148572,1.150852,1.529600,38.743733,5.000892,2.227969,3.830483,1.354447,0.022283,1.064131
YAL004W,5.885958,5.035586,23.627768,0.280726,1.793227,3.916185,0.407873,0.662228,0.274522,0.141925,1.052574,1.165382,27.583021,2.375404,2.284112,3.952990,1.323046,0.040681,3.180600
YAL005C,5.903218,4.826971,23.024489,0.300006,1.846948,4.764524,0.410972,0.707888,0.312808,0.136524,1.121279,1.149201,22.608953,2.697176,2.152940,3.711750,1.367357,0.043073,2.724967
YAL007C,5.882356,5.405774,20.391227,0.258860,1.884296,5.554489,0.357562,0.946165,0.283098,0.093918,1.289670,1.161532,28.793732,2.875816,2.414602,3.967596,1.231819,0.038839,2.659337
YAL008W,5.838249,4.682146,19.302517,0.286509,2.366559,7.124882,0.424865,1.244891,0.352115,0.108243,1.224425,1.125802,26.754974,4.704578,2.784608,3.851613,1.204111,0.055207,1.465688


## Subset to the genes currently in SGD

In [25]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [26]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [27]:
data.head()

,dataset_id,11788,11789,11790,11791,11792,11793,11794,11795,11796,11797,11798,11799,11800,11801,11802,11803,11804,11805,11806
,data_type,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,,,,,,,,
2,YAL002W,7.575695,6.023942,29.799218,0.172805,1.668835,5.518672,0.346954,1.017763,0.140872,0.148572,1.150852,1.529600,38.743733,5.000892,2.227969,3.830483,1.354447,0.022283,1.064131
1863,YAL004W,5.885958,5.035586,23.627768,0.280726,1.793227,3.916185,0.407873,0.662228,0.274522,0.141925,1.052574,1.165382,27.583021,2.375404,2.284112,3.952990,1.323046,0.040681,3.180600
4,YAL005C,5.903218,4.826971,23.024489,0.300006,1.846948,4.764524,0.410972,0.707888,0.312808,0.136524,1.121279,1.149201,22.608953,2.697176,2.152940,3.711750,1.367357,0.043073,2.724967
5,YAL007C,5.882356,5.405774,20.391227,0.258860,1.884296,5.554489,0.357562,0.946165,0.283098,0.093918,1.289670,1.161532,28.793732,2.875816,2.414602,3.967596,1.231819,0.038839,2.659337
6,YAL008W,5.838249,4.682146,19.302517,0.286509,2.366559,7.124882,0.424865,1.244891,0.352115,0.108243,1.224425,1.125802,26.754974,4.704578,2.784608,3.851613,1.204111,0.055207,1.465688


# Normalize

In [28]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [29]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [30]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [31]:
data_all.head()


dataset_id          11788     11789      11790     11791     11792     11793  \
data_type           value     value      value     value     value     value   
gene_id orf                                                                    
2       YAL002W  7.575695  6.023942  29.799218  0.172805  1.668835  5.518672   
1863    YAL004W  5.885958  5.035586  23.627768  0.280726  1.793227  3.916185   
4       YAL005C  5.903218  4.826971  23.024489  0.300006  1.846948  4.764524   
5       YAL007C  5.882356  5.405774  20.391227  0.258860  1.884296  5.554489   
6       YAL008W  5.838249  4.682146  19.302517  0.286509  2.366559  7.124882   

dataset_id          11794     11795     11796     11797  ...            \
data_type           value     value     value     value  ...    valuez   
gene_id orf                                              ...             
2       YAL002W  0.346954  1.017763  0.140872  0.148572  ...  1.037930   
1863    YAL004W  0.407873  0.662228  0.274522  0.141925  ...  0.748155   
4       YAL005C  0.410972  0.707888  0.312808  0.136524  ...  0.512700   
5       YAL007C  0.357562  0.946165  0.283098  0.093918  ... -1.344725   
6       YAL008W  0.424865  1.244891  0.352115  0.108243  ... -0.720245   

dataset_id          11798     11799     11800     11801     11802     11803  \
data_type          valuez    valuez    valuez    valuez    valuez    valuez   
gene_id orf                                                                   
2       YAL002W -0.522143  2.902889  2.732851  1.455113 -0.329635 -0.532187   
1863    YAL004W -1.223069  0.240776 -0.171282 -0.236886 -0.153460 -0.181425   
4       YAL005C -0.733057  0.122508 -1.465586 -0.029520 -0.565070 -0.872144   
5       YAL007C  0.467925  0.212641  0.143757  0.085605  0.256014 -0.139605   
6       YAL008W  0.002594 -0.048516 -0.386749  1.264153  1.417077 -0.471688   

dataset_id          11804     11805     11806  
data_type          valuez    valuez    valuez  
gene_id orf                                    
2       YAL002W  0.054165 -2.418802 -1.653385  
1863    YAL004W -0.073077  0.247957  0.582325  
4       YAL005C  0.106478  0.594623  0.101021  
5       YAL007C -0.442743 -0.018994  0.031694  
6       YAL008W -0.555018  2.353355 -1.229204  

[5 rows x 38 columns]

# Print out

In [32]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [33]:
from IO.save_data_to_db3 import *

In [34]:
save_data_to_db(data_all, paper_pmid)

Deleting all datasets for PMID 27693354...


  0%|          | 0/19 [00:00<?, ?it/s]

Inserting the new data...


100%|██████████| 19/19 [02:18<00:00,  7.27s/it]

Updating the data_modified_on field...
